In [1]:
# import all modules here

import numpy as np
import os
import pickle

from data import load_data
from visualiser import visualize_images, plot_class_frequencies, convert_to_grayscale
from feature_ import l_features, extract_features, get_from_gpu
from LwP_ import LwP, LwP_updated
from sampler import sampler

import torch
import warnings
warnings.filterwarnings('ignore')

In [2]:
model = LwP(10)
initial_img, initial_label = load_data(1, 'train', 'one')
X_train = l_features(1, 'train', 'one')
model.fit(X_train, initial_label)

In [3]:
X_eval = l_features(1, 'eval', 'one')
y_eval = load_data(1, 'eval', 'one', y_only=True)
y_pred = model.predict(X_eval)
print(np.mean(y_eval == y_pred))

0.8536


In [5]:
for i in range(2, 11): # for part two
    print(f'Running {i}')

    X = l_features(i, 'train', 'one')
    im = load_data(i, 'train', 'one')

    X_opt, y_opt = model.top_k(X, 50*(i - 1), indices=True)
    X_new = np.asanyarray([im[i] for i in X_opt])
    X_new = ((torch.tensor(X_new)).float() / 255.0).permute(0, 3, 1, 2).cuda()
    X_gen, y_gen = sampler().generate(X_new, torch.tensor(y_opt), 1)
    X_gen = extract_features(X_gen)

    # X_new, y_new = model2.top_k(X_new, 10*(i - 1))
    if (i != 1): model.update(X_gen, y_gen, True)

    for j in range(1, i + 1):
        y = load_data(j, 'eval', 'one', y_only=True)
        X = l_features(j, 'eval', 'one')
        y_pred = model.predict(X, 'euc')

        acc = np.mean(y_pred == y)
        # res[i - 1][j - 1] = f'{acc * 100:.2f}'
        print(f'{acc * 100:.2f}', end=' ')

    print()

Running 2
78.44 77.32 
Running 3
74.88 

KeyboardInterrupt: 